In [1]:
from sympy import *
from sympy import I as j #import complex number as j
from sympy.solvers.solveset import linsolve, nonlinsolve
from sympy.parsing.latex import parse_latex
from sympy.parsing.sympy_parser import parse_expr
from IPython.display import display, Math, HTML
import ipywidgets as widgets

#for plotting
from numpy import linspace
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

#define symbols
Res = symbols("Res")
R=symbols("R_{0:7}", real = True)
I=symbols("I_{0:7}")
i=symbols("i_{0:7}")
C=symbols("C_{0:7}", real=True)
L=symbols("L_{0:7}")
V=symbols("V_{0:7}")
Vin = symbols("V_in")
Vr = symbols("V_i*n")
t =symbols("t", real=True)
s = symbols("s")
a, b  = symbols("a b")
omega=symbols("omega")
#j=symbols("j")
Pr3 = symbols("P_R3")

#udeful SI prefix dict
prefix = {'y': 1e-24,  # yocto
           'z': 1e-21,  # zepto
           'a': 1e-18,  # atto
           'f': 1e-15,  # femto
           'p': 1e-12,  # pico
           'n': 1e-9,   # nano
           'u': 1e-6,   # micro
           'm': 1e-3,   # mili
           'c': 1e-2,   # centi
           'd': 1e-1,   # deci
           '' : 1,      # nothing
           'k': 1e3,    # kilo
           'M': 1e6,    # mega
           'G': 1e9,    # giga
           'T': 1e12,   # tera
           'P': 1e15,   # peta
           'E': 1e18,   # exa
           'Z': 1e21,   # zetta
           'Y': 1e24,   # yotta
    }

In [2]:

out = widgets.Output(layout={'border': '1px solid black'})
outIPy = widgets.Output(layout={'border': '2px solid black'})
initNumOfBoxes = {"new" : 2}
numOfR = 5
textBox = []
subsEqNum = 1

prsEq = []
linEq = []
lapEq = []
cEq = []
#---- plotting
mpl.rcParams['text.usetex'] = True
#amsmath needed for \text in equations and bm for \bm bold text
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}\usepackage{bm}'

#floor with precision
def my_floor(a, precision=0):
    return np.true_divide(np.floor(a * 10**precision), 10**precision)
#ceil with precision
def my_ceil(a, precision=0):
    return np.true_divide(np.ceil(a * 10**precision), 10**precision)


@outIPy.capture()  #capture the plot output
def prettyPlot(expr, x_symb, x_start, x_end, x_inc, si_prefix, x_title="", y_title="", x_unit="", y_unit="", y_symb="", legend="True"):
    global lam_x, x_vals, y_vals
    
    def diracDeltaArray(x):
        #resultArray = []
        #for i, val in np.ndenumerate(x):
        #    resultArray.append(DiracDelta(val))

        #return resultArray     
        if type(x) == int:
            return 0
        else:
            return np.zeros(len(x))
            
    fig = plt.figure(figsize=(8, 4), dpi=80)
    ax = plt.gca()
    
    symExpr = expr #atan((740*pi)/(1000+R))
    
    symExpr = parse_expr(str(symExpr.subs("e", E)).replace("theta", "Heaviside").replace("delta", "DiracDelta")) #theta does'nt plot properly
    
    #display(symExpr)
    lam_x = lambdify(t, symExpr, modules=[{"DiracDelta": diracDeltaArray}, 'numpy'])

    #vect_x = np.vectorize(lam_x)

    x_vals = np.arange(x_start, x_end+x_inc, x_inc)
    y_vals = lam_x(x_vals*prefix[si_prefix])

    plt.plot(x_vals, y_vals)
    plt.xlabel(x_title + latex(x_symb) + ", " + si_prefix + x_unit)
    #plt.ylabel(y_title + latex(y_symb) + latex(", " + si_prefix) + x_unit)
    plt.ylabel(r"$\bm{" + y_title + y_symb + "}$" + ", " + y_unit)
    #plt.ylabel(r"Fāze $\bm{\varphi}$, °")

    #plt.yticks(np.arange(my_floor(min(y_vals), -1), my_ceil(max(y_vals), -1)+1, 10)) may bee needed for some functions
    
    if legend == True:
        leg = plt.legend(['$' + latex(symExpr) + '$'], framealpha = 1, prop={'size': 12})

    sns.set_style("whitegrid")
    sns.set_context("talk")
    #set x and y minor tick locations
    ax.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator())
    ax.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator())
    ax.grid(b=True, which='major', color='dimgrey', linewidth=2.0)
    ax.grid(b=True , which='minor', axis='both', color='grey', linestyle='dotted', linewidth=0.5)
    plt.show()
    
# plotting ---


style = {'description_width': 'initial'}
layout = widgets.Layout(width='auto', height='40px')

def drawTBox(change): #dinamically draw prsEq input text boxes
    out.clear_output()
    
    global numOfBoxes   #global needed because without it the next statement wouldn't update the global variable
    numOfBoxes = int(change["new"])
    
    global textBox
    textBox = []
    for i in range(numOfBoxes):
        textString = "Eq" + str(i+1) + ":"
        textBox.append(widgets.Text(description=textString))
        #print(numOfBoxes)
        with out:
            display(textBox[i])

drawTBox(initNumOfBoxes)

dropdown = widgets.Dropdown(
    options=['1', '2', '3', '4'],
    value = str(initNumOfBoxes["new"]),
    description='Number of prsEqs:',
    disabled=False,
    style=style
)

#def tes(change):
#    print(change["new"])

def changeSubs(change):  #change for which prsEqs to substitute values
    global subsEqNum
    subsEqNum = change["new"]

#drawTBox(int(dropdown.value))

radioButton = widgets.RadioButtons(
    options=[('parsed equation',1), ('linsolve equation',2), ('custom equation',3)],
    disabled=False,
    layout=layout
)

radioLabel = widgets.Label('Substitute or inv. laplace transform:', layout=widgets.Layout(width='auto'))

checkBox1 = widgets.Checkbox(
    value=False,
    description='Legend'
)

button1 = widgets.Button(
    description='Parse equations',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)

button2 = widgets.Button(
    description='LinSolve for I',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)

button3 = widgets.Button(
    description='Substitute in equations',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button4 = widgets.Button(
    description='Clear Subs Val',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button5 = widgets.Button(
    description='Clear Output',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button6 = widgets.Button(
    description='Inverse Laplace Trans',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button7 = widgets.Button(
    description='Calculate',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button8 = widgets.Button(
    description='Graph',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button9 = widgets.Button(
    description='Solve for V',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button10 = widgets.Button(
    description='Eval',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

#prsEq = [Eq((R[1]+1/(C[1]*s))*I[1]-(1/(C[1]*s))*I[2],V[1]),
#           Eq(-1/(C[1]*s)*I[1]+(R[2]+1/(C[1]*s))*I[2],0)]
#display(prsEq[0])
#for i in prsEq:
 #   display(i)
#(R[1]+1/(C[1]*s))*I[1]-(1/(C[1]*s))*I[2]

#def handle_submit(sender):
 #   print(textBox.value)
  #  print(sender)

def handle_button8(a): # graph equations
    boxInLhs = customEqBox.value
    if customEqBox.value:
        
        for key, val in {"prsEq":prsEq, "linEq":linEq, "lapEq":lapEq, "cEq":cEq}.items():
            
            for num in range(len(val)):
        
                boxInLhsNew = boxInLhs.find(key + "[" + str(num) + "]")
                
                
                if boxInLhsNew >= 0:
                    
                    boxInLhsNew = val[num]
                    
                    break
            else:
                continue
            break
                
        prettyPlot(boxInLhsNew.rhs, t, 0, 2, 0.001, si_prefix="m", x_unit="s", y_unit="V", y_symb="v_{ex}", legend=checkBox1.value)

    
def handle_button7(but10=False):  #custom equation calc
    global cEq
    cEq = []
    boxInLhs = customEqBox.value
    boxInRhs = boxInLhs
    
    if customEqBox.value:
        
        for key, val in {"prsEq":prsEq, "linEq":linEq, "lapEq":lapEq}.items():
            
            for num in range(len(val)):
                
                if str(type(boxInLhs)) != "<class 'sympy.core.relational.Equality'>":
                    val[num] = Eq(Res, val[num])
                
                prsEqLhs = str(val[num].lhs).replace("{", "").replace("}", " ") #because parse_expre dosen't like I_{x}
                prsEqRhs = str(val[num].rhs).replace("{", "").replace("}", " ")
                boxInLhs = boxInLhs.replace(key + "[" + str(num) + "]", prsEqLhs)
                boxInRhs = boxInRhs.replace(key + "[" + str(num) + "]", prsEqRhs)
            
        cEq.append(Eq(parse_expr(boxInLhs), parse_expr(boxInRhs)))
    
    #elif boxIn.find("linEq[")
    #for n in [prsEq] 
    
    if but10: 
        for eq in cEq:
            outIPy.append_display_data(eq.evalf())
    else:
        for eq in cEq:
            outIPy.append_display_data(eq)

def handle_button10(a):
    handle_button7(but10=True)
            
def handle_button6(a):  #laplace transform the solved linEqs 
    global lapEq
    lapEq = []
    if subsEqNum == 1: #for parsed prsEqs
        eqNum = 0

        #print(subsEqNum)
    elif subsEqNum == 2: #for linsolve linEqs
        eqNum = 1

    elif subsEqNum == 3: #for custom equations
        eqNum = 2
        
    for n, res in enumerate([prsEq, linEq, cEq][eqNum]):
        lapEq.append(Eq(i[n+1], expand(inverse_laplace_transform(res.rhs,s,t)))) #inv laplace dosent work for equalities
        outIPy.append_display_data(lapEq[n])


def handle_button5(a):  #clear output
    outIPy.clear_output()
    
#def handle_button4(a): #clear substitute input field values

def handle_button3(a):  #substitute values parser
    global prsEq
    if subsEqNum == 3:
        forRange = range(len(cEq))
    else:
        forRange = range(len(prsEq))
        
    for nEq in forRange:
        for key, textField in valueItems.items():
            for num, textF in enumerate(textField):
                if textF.value != "":
                    #print(key)
                    #print(textF.value)
                    if key=="L":
                        mult = 10**-3
                    elif key=="C":
                        mult = 10**-6 
                    else:
                        mult = 1
                    #print(key + "_{" + str(n) + "}")
                    #print(int(textF.value))
                    
                    bnum = "{" + str(num) + "}" # because Vin and custom eqs dosent need the braces {}
                    if key=="Vin":
                        bnum = "in"
                        key = "V"
                    
                    if subsEqNum == 1: #for parsed prsEqs
                        prsEq[nEq] = prsEq[nEq].subs(key + "_" + str(bnum), parse_expr(textF.value)*mult)
                        
                        #print(subsEqNum)
                    elif subsEqNum == 2: #for linsolve linEqs
                        linEq[nEq] = linEq[nEq].subs(key + "_" + str(bnum), parse_expr(textF.value)*mult)
                        
                    elif subsEqNum == 3: #for custom equations
                        cEq[nEq] = cEq[nEq].subs(key + "_" + str(num), parse_expr(textF.value)*mult)
                        display()
                        #print(subsEqNum)
            
        if subsEqNum == 1: #for parsed prsEqs
            outIPy.append_display_data(prsEq[nEq])
        elif subsEqNum == 2: #for linsolve linEqs
            outIPy.append_display_data(linEq[nEq])
        elif subsEqNum == 3: #for custom equations
            outIPy.append_display_data(cEq[nEq])
                    #print(eq)
    #display(prsEq[0])

def handle_button9(a):  #linear solver for V
    vList = []
    global linEq
    for n, eq in enumerate(prsEq):
        vList.append(V[n+1])
        
    linEq = linsolve(prsEq,vList).args[0]
    linEq = list(linEq)  #needed as list in other functions
    #outIPy.append_display_data(type(linEq))
    outIPy.append_display_data("result:")
    for n, res in enumerate(linEq):
        linEq[n] = Eq(V[n+1], linEq[n])
        outIPy.append_display_data(Math(latex(linEq[n]))) #pretty print
    
def handle_button2(a):  #linear solver
    iList = []
    global linEq
    for n, eq in enumerate(prsEq):
        iList.append(I[n+1])
        
    linEq = linsolve(prsEq,iList).args[0]
    linEq = list(linEq)  #needed as list in other functions
    #outIPy.append_display_data(type(linEq))
    outIPy.append_display_data("result:")
    for n, res in enumerate(linEq):
        linEq[n] = Eq(I[n+1], linEq[n])
        outIPy.append_display_data(Math(latex(linEq[n]))) #pretty print
    
    
def handle_button1(a):  #input parser
    global prsEq
    latex_string = []
    prsEq = []

    for i in range(numOfBoxes):
        lstring = textBox[i].value.replace(r"\ ", "") # remove blank spaces
        lstring = textBox[i].value.replace("[", "{").replace("]", "}") # replace square brackets, because the interpreter dsnt understand them
        lstring = parse_latex(lstring)
        #lstring = lstring.subs(j, i)
        lstring = lstring.subs("V_{i*n}", Vin) # multiple charachter subscripts are parsed incorectly
        prsEq.append(lstring)
        outIPy.append_display_data(prsEq[i])
        #display(type(prsEq[i]))
    prsEq = list(prsEq)
        
valueItems = {"R": [], "C": [], "L": [], "V": [],"Vin": [], "I": []}
valueItemPlaceholder = {"R": "Ohms", "C": "uF", "L": "mH", "V": "Volts","Vin": "input eq", "I": "Amps"}
valueChildList = []
for key in valueItems:
    if key=="Vin":
        oldNumOfR = numOfR
        numOfR = 1
    for n in range(numOfR):
        if key=="Vin":
            n = "{in}"
            valueItems[key].append(widgets.Text(description="$" + "V" + "_" + str(n) + "$", placeholder=valueItemPlaceholder[key]))
        else:
            valueItems[key].append(widgets.Text(description="$" + key + "_" + str(n) + "$", placeholder=valueItemPlaceholder[key]))
    valueChildList.append(widgets.VBox(valueItems[key]))
    if key=="Vin":
        numOfR = oldNumOfR
    
box_layout = widgets.Layout(overflow='scroll hidden',
                    border='3px solid black',
                    width='500px',
                    height='',
                    flex_flow='column',
                    display='flex')
#carousel = widgets.Box(children=valueItems, layout=box_layout)
#carousel = widgets.Box(children=valueItems)

display(dropdown)
dropdown.observe(drawTBox, names="value")

display(widgets.Box(valueChildList))

display(out)
display(widgets.HBox([button1,button2,button9,button6,button3,button4,button5]))

display("Write custom expression:  prsEq[x] parsed   linEq[x] system of equation results   lapEq[x] inv. laplace results")
display("cEq[x] custom eq (only for graphing)")

customEqBox = widgets.Text(placeholder="Equation")
display(widgets.HBox([customEqBox, button7, button8, button10, checkBox1]))
display(widgets.HBox([widgets.Text(placeholder="Paste here <<Ctrl+v>>"),widgets.Text(placeholder="or here <<Ctrl+v>>")]))

display(widgets.HBox([radioLabel, radioButton], layout=widgets.Layout(padding = "0px 0px 20px 0px")))
radioButton.observe(changeSubs, names="value")

display(outIPy)

button1.on_click(handle_button1)
button2.on_click(handle_button2)
button3.on_click(handle_button3)
#button4.on_click(handle_button4)
button5.on_click(handle_button5)
button6.on_click(handle_button6)
button7.on_click(handle_button7)
button8.on_click(handle_button8)
button9.on_click(handle_button9)
button10.on_click(handle_button10)

Dropdown(description='Number of pEqs:', index=1, options=('1', '2', '3', '4'), style=DescriptionStyle(descript…

Box(children=(VBox(children=(Text(value='', description='$R_0$', placeholder='Ohms'), Text(value='', descripti…

Output(layout=Layout(border='1px solid black'))

'Write custom expression:  pEq[x] parsed   linEq[x] system of equation results   lapEq[x] inv. laplace results'

Output(layout=Layout(border='2px solid black'))